In [ ]:
import json
import re
import unicodedata
import pandas as pd
from pathlib import Path

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Arabic to SQL Data/Text To SQL Task-20250223T185920Z-001.zip" -d "/content/dataset"

Archive:  /content/drive/MyDrive/Arabic to SQL Data/Text To SQL Task-20250223T185920Z-001.zip
  inflating: /content/dataset/Text To SQL Task/Text to SQL.docx  
  inflating: /content/dataset/Text To SQL Task/Papers/Survey of LLM-based Text-to-SQL.pdf  
  inflating: /content/dataset/Text To SQL Task/Papers/Text-to-SQL in Arabic.pdf  
  inflating: /content/dataset/Text To SQL Task/Papers/deep learning survey.pdf  
  inflating: /content/dataset/Text To SQL Task/Dataset/database/coffee_shop/coffee_shop.sqlite  
  inflating: /content/dataset/Text To SQL Task/Dataset/database/company_employee/schema.sql  
  inflating: /content/dataset/Text To SQL Task/Dataset/database/apartment_rentals/apartment_rentals.sqlite  
  inflating: /content/dataset/Text To SQL Task/Dataset/database/cinema/schema.sql  
  inflating: /content/dataset/Text To SQL Task/Dataset/database/college_1/TinyCollege.sql  
  inflating: /content/dataset/Text To SQL Task/Dataset/database/academic/schema.sql  
  inflating: /content/d

In [ ]:
jsonl_path = "/content/dataset/Text To SQL Task/Dataset/AR_spider.jsonl"
data = []

In [ ]:
with open(jsonl_path, "r", encoding="utf-8") as file:
    for line in file:
        entry = json.loads(line.strip())
        data.append({
            "arabic": entry["arabic"],
            "sql": entry["query"],
            "db_id": entry["db_id"]
        })

In [ ]:
def normalize_arabic(text):
    """Normalize Arabic text by removing diacritics and normalizing characters."""
    text = unicodedata.normalize("NFKC", text)  # Unicode normalization
    text = re.sub(r"[\u064B-\u065F]", "", text)  # Remove diacritics
    text = text.replace("ى", "ي").replace("ة", "ه")  # Normalize letters
    return text.strip()

In [ ]:
def tokenize_sql(query):
    """Ensure SQL queries are tokenized properly by spacing special characters."""
    query = re.sub(r"([,()=<>])", r" \1 ", query)
    query = re.sub(r"\s+", " ", query)
    return query.strip()

In [ ]:
processed_data = []
for entry in data:
    processed_data.append({
        "arabic": normalize_arabic(entry["arabic"]),
        "sql": tokenize_sql(entry["sql"]),
        "db_id": entry["db_id"]
    })

In [ ]:
output_df = pd.DataFrame(processed_data)
output_df.to_csv("processed_arabic_sql.csv", index=False, encoding="utf-8")

print("Preprocessing complete. Data saved to processed_arabic_sql.csv.")

Preprocessing complete. Data saved to processed_arabic_sql.csv.


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import MBartForConditionalGeneration, MBartTokenizer, Trainer, TrainingArguments
from transformers import MBart50Tokenizer

In [ ]:
df = pd.read_csv("/content/processed_arabic_sql.csv")

In [ ]:
def preprocess_function(examples, tokenizer):
    """Tokenize Arabic text (input) and SQL query (target)."""
    model_inputs = tokenizer(examples["arabic"], padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["sql"], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
raw_datasets = Dataset.from_pandas(df)
model_name = "facebook/mbart-large-50"
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="ar_AR", tgt_lang="en_XX")
tokenized_datasets = raw_datasets.map(lambda x: preprocess_function(x, tokenizer), batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Map:   0%|          | 0/6396 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
model = MBartForConditionalGeneration.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./mbart_arabic_sql",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-17-6abd1c8e097b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.641300,0.131037
2,0.113800,0.059727
3,0.072100,0.035677


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2400, training_loss=0.42770745992660525, metrics={'train_runtime': 4742.8227, 'train_samples_per_second': 4.046, 'train_steps_per_second': 0.506, 'total_flos': 5197860748394496.0, 'train_loss': 0.42770745992660525, 'epoch': 3.0})

In [ ]:
trainer.save_model("./mbart_arabic_sql_finetuned")
tokenizer.save_pretrained("./mbart_arabic_sql_finetuned")

('./mbart_arabic_sql_finetuned/tokenizer_config.json',
 './mbart_arabic_sql_finetuned/special_tokens_map.json',
 './mbart_arabic_sql_finetuned/sentencepiece.bpe.model',
 './mbart_arabic_sql_finetuned/added_tokens.json')

In [ ]:
import shutil

shutil.make_archive('/content/mbart_arabic_sql_finetuned', 'zip', '/content/mbart_arabic_sql_finetuned')

'/content/mbart_arabic_sql_finetuned.zip'

In [ ]:
!cp -r mbart_arabic_sql_finetuned /content/drive/MyDrive/

#Testing

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [ ]:
model_path = "/content/drive/MyDrive/mbart_arabic_sql_finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=

In [ ]:
def arabic_to_sql(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
question_ar = "ما هي أسماء الطلاب الذين حصلوا على درجات أعلى من 90؟"
sql = arabic_to_sql(question_ar)
print("Arabic Question:", question_ar)
print("Generated SQL:", sql)


Arabic Question: ما هي أسماء الطلاب الذين حصلوا على درجات أعلى من 90؟
Generated SQL: SELECT T1.stu_fname FROM student AS T1 JOIN enroll AS T2 ON T1.stu_num = T2.stu_num WHERE enroll_grade = 'C'


In [ ]:
question_ar = "كم عدد رؤساء الأقسام الذين تزيد أعمارهم عن 56 سنه؟"
sql = arabic_to_sql(question_ar)
print("Arabic Question:", question_ar)
print("Generated SQL:", sql)


Arabic Question: كم عدد رؤساء الأقسام الذين تزيد أعمارهم عن 56 سنه؟
Generated SQL: SELECT count ( * ) FROM head WHERE age > 56


In [ ]:
question_ar = "ما هو رقم تعريفي التقسيم للمستخدم الذي يحمل اسم آيرون مان"
sql = arabic_to_sql(question_ar)
print("Arabic Question:", question_ar)
print("Generated SQL:", sql)


Arabic Question: ما هو رقم تعريفي التقسيم للمستخدم الذي يحمل اسم آيرون مان
Generated SQL: SELECT count ( * ) FROM user_profiles WHERE name = 'Alice'


In [ ]:
question_ar = "ما هو اسم الإدخال لأغلي فهرس (بالدولار الأمريكي)؟"
sql = arabic_to_sql(question_ar)
print("Arabic Question:", question_ar)
print("Generated SQL:", sql)


Arabic Question: ما هو اسم الإدخال لأغلي فهرس (بالدولار الأمريكي)؟
Generated SQL: SELECT catalog_entry_name FROM catalog_contents ORDER BY price_in_dollars DESC LIMIT 1


In [ ]:
question_ar = "ما هي المدن المختلفه التي يعيش فيها الأشخاص؟"
sql = arabic_to_sql(question_ar)
print("Arabic Question:", question_ar)
print("Generated SQL:", sql)

Arabic Question: ما هي المدن المختلفه التي يعيش فيها الأشخاص؟
Generated SQL: SELECT DISTINCT city FROM people


In [ ]:
question_ar = "لكل مدينه، ماهو أعلي خط عرض بين محطاتها؟"
sql = arabic_to_sql(question_ar)
print("Arabic Question:", question_ar)
print("Generated SQL:", sql)

Arabic Question: لكل مدينه، ماهو أعلي خط عرض بين محطاتها؟
Generated SQL: SELECT city , max ( lat ) FROM station GROUP BY city


In [ ]:
question_ar = "كم عدد المحطات في ماونتن فيو"
sql = arabic_to_sql(question_ar)
print("Arabic Question:", question_ar)
print("Generated SQL:", sql)

Arabic Question: كم عدد المحطات في ماونتن فيو
Generated SQL: SELECT count ( * ) FROM station WHERE city = "Mountain View"


In [ ]:
question_ar = "ما هي أسماء الطلاب الذين حصلوا على درجات اقل من 90؟"
sql = arabic_to_sql(question_ar)
print("Arabic Question:", question_ar)
print("Generated SQL:", sql)

Arabic Question: ما هي أسماء الطلاب الذين حصلوا على درجات اقل من 90؟
Generated SQL: SELECT T2.name FROM student AS T1 JOIN enroll AS T2 ON T1.stuid = T2.stuid JOIN CLASS AS T3 ON T1.class_code = T3.class_code WHERE T3.crs_code = 'ACCT-211'
